# Impact parameter for random lines in square lattice

*Christopher Tunnell tunnell@rice.edu*
*Charles Dyall crd15@rice.edu*

This notebook computes the [impact parameter](https://en.wikipedia.org/wiki/Impact_parameter) between nodes of a cubic lattice and random lines.  The purpose of this notebook is to demonstrate for Windchime that a few nodes will have significantly smaller impact parameters relative to the lattice spacing, which becomes even more advantageous when factoring in the $r^2$ scaling of gravitational attraction.

Based on idea originally from Juehang Qin (Purdue).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

## Experimental setup

Set these parameters of the simulation.

In [2]:
# There are two free parameters.  The length
# scale L, where the cube goes from [-L/2, L/2] in
# each dimension.  Then there are 'n' sensors along
# this axis.
L = 1. # m
n = 1001 # number of sensors

# We will compute the average impact factor for
# 'lines' number of random lines.
lines = 100

_No need to modify below here if you are just running the code and not developing_

In [3]:
x = y = z = np.linspace(-L/2, L/2, n)
spacing = x[1] - x[0]
print(f'Spacing {spacing:0.3g} m')

Spacing 0.01 m


In [4]:
# These contain the 3D coordinates of each node.
X, Y, Z = np.meshgrid(x, y, z,
                      indexing='ij')

# Lattice nodes in shape of (3,n) for each of computation
points = np.stack((X.ravel(), Y.ravel(), Z.ravel())).T

# This will be distances, aka impact parameter
D = np.zeros_like(X)

## Compute distances of sensors to random line

The first step will be to compute the distance from random lines (defined by two random points on cube boundary) to the sensors.

![DrawingCoord](PointLineDistance3D_1000.gif)

In [5]:
def shuffle_along_axis(a, axis):
    # This shuffles randomly x, y, z positions in an
    # array of positions.  This is for generating random
    # positions later.
    idx = np.random.rand(*a.shape).argsort(axis=axis)
    return np.take_along_axis(a,idx,axis=axis)

In [6]:
def random_point_cube(n = 5):
    # Pick a random point on the surface of a cube.
    
    # Pick side of cube
    s = np.random.randint(6, size=n)

    # One axis is just the face distance
    # (Warning: this sets everything to L)
    result = np.random.choice((L/2, -L/2), (n, 3))

    # Then pick random point on that face
    result[:, 1:] = (np.random.random((n,2)) - 0.5) * L

    return shuffle_along_axis(result, axis=1)

#random_point_cube(2)

In [7]:
# This is used for defining our own cross product
eijk = np.zeros((3, 3, 3))
eijk[0, 1, 2] = eijk[1, 2, 0] = eijk[2, 0, 1] = 1
eijk[0, 2, 1] = eijk[2, 1, 0] = eijk[1, 0, 2] = -1

def cross_array(a, b):
    # Returns cross product of a[] and b[] produces a[i]xb[i]
    return np.einsum('ijk,aj,ak->ai', eijk,
                     a, b)

# Check
a = np.random.random(((5, 3)))
b = np.random.random(((5, 3)))
assert (np.cross(a[0], b[0]) == cross_array(a, b)[0]).all()

In [8]:
def distance2(x0, x1, x2):
    # Determine impact parameter
    # Distance of point x0[] from line formed by x1, x2
    # https://mathworld.wolfram.com/Point-LineDistance3-Dimensional.html
    return np.linalg.norm(cross_array((x0 - x1),
                                   (x0 - x2)),
                         axis=1) / np.linalg.norm(x2 - x1)

In [9]:
assert 0 == distance2(np.array(((0, 0, 0), (0,0,0))),
                     np.array((0, 0, 0)),
                     np.array((0, 0, 1)))[0]
assert 1 == distance2(np.array(((0, 1, 0), (0,0,0))),
                     np.array((0, 0, 0)),
                     np.array((0, 0, 1)))[0]

In [10]:
Gn = 6.67430e-11 # m^3 kg^-1 s^-2, https://en.wikipedia.org/wiki/Newton%27s_law_of_universal_gravitation

def get_impulse(b,
                Msensor = 1e-3, # kg, i.e. a gram
                Mdm = 2.176434e-8, # kg https://en.wikipedia.org/wiki/Planck_units
                v = 220000, # m/s, from memory
               ):
    # Get impulse (momentum change) b[] impact parameter
    # Eq. 2 of https://arxiv.org/pdf/1903.00492.pdf
    # Integrate[Gn * Mx * Ms * b / (b^2 + v^2 * x^2)^(3/2),{x, -Infinity, Infinity}]
    return (2 * b * Gn * Msensor * Mdm)/(b**2)**(3/2) * np.sqrt(v**2/b**2)


We bin for the computation so we are not storing tons of distances for each position. This is more memory efficient since we are able to simulate so many realizations of the detector.

In [11]:
number_of_bins = 200 # many bins, can always reduce

max_l = np.sqrt(3) * 2 * L

bins_b = np.logspace(-15,
                     np.log10(max_l),
                     number_of_bins)
data_b = np.zeros_like(bins_b[1:])

bins_b2 = np.logspace(np.log10((max_l)**-2),
                      15,
                     number_of_bins)
data_b2 = np.zeros_like(bins_b2[1:])

bins_J = np.logspace(-20,
                      3,
                     number_of_bins)
data_J = np.zeros_like(bins_J[1:])


# How many times did we simulate.  This means you can
# run the computation incrementally by rerunning block
# below.
computed_n = 0 

### The computation loop

You can rerun this.  Here we integrate over random lines.

In [ ]:
# Create pairs of random 3D points on cube that
# define the line.
line_seeds = random_point_cube(n=2*lines).reshape(lines,
                                                  2,
                                                  3)

for x1, x2 in tqdm(line_seeds):
    # x1 and x2 are points defining random line
    
    # Here is the computation in 3 steps
    # result etc is vector with one element per sensor
    result = distance2(points, # sensor locations
                       x1, x2, # this is the line
                      )
    result2 = result**-2
    result3 = get_impulse(result)
        
    data_b += np.histogram(result,
                           bins=bins_b)[0]/lines
    data_b2 += np.histogram(result2,
                            bins=bins_b2)[0]/lines
    data_J += np.histogram(get_impulse(result),
                           bins=bins_J)[0]/lines
    
    # Some sanity checks to check results in bin bounds
    assert bins_b[0] < result.min(), result.min()
    assert bins_b[-1] > result.max(), result.max()
    assert bins_b2[0] < result2.min(), result2.min()
    assert bins_b2[-1] > result2.max(), result2.max()
    assert bins_J[0] < result3.min(), np.log10(result3.min())
    assert bins_J[-1] > result3.max(), np.log10(result3.max())
    
computed_n += lines

  2%|▏         | 2/100 [15:11<12:23:51, 455.42s/it]

In [ ]:
filename = 'windchime_cache.npz'

In [ ]:
np.savez(open(filename, 'wb'),
         bins_b=bins_b, data_b=data_b,
         bins_b2=bins_b2, data_b2=data_b2,
         bins_J=bins_J, data_J=data_J,
         computed_n=computed_n)

In [ ]:
with np.load(filename) as file:
    bins_b = file['bins_b']
    data_b = file['data_b']
    bins_b2 = file['bins_b2']
    data_b2 = file['data_b2']
    bins_J = file['bins_J']
    data_J = file['data_J']
    computed_n = file['computed_n']

In [ ]:
def get_nonzero_xlim(data, bins):
    # This function just identifies left and right
    # bounds to suppress zero bins on either side.
    return (bins[np.argmax(data > 0)],
            bins[-1*np.argmax(np.flip(data) > 0)])

## Plot impact parameter

The impact parameter is $b$

In [ ]:
fig, ax = plt.subplots()
ax.bar(bins_b[:-1],
       data_b,
       width=np.diff(bins_b),
       align="edge")

plt.loglog()
plt.axvline(spacing, label='$s$', color='red')
plt.axvline(L, label='$L$', color='black')
plt.legend()
plt.xlabel('Impact parameter $b$ [m]')
plt.ylabel('Counts per line [arb.]')
plt.axhline(1, color='yellow', ls='--')
plt.xlim(*get_nonzero_xlim(data_b, bins_b))
plt.text(0.05, 0.35, f'Lattice side $L$ = {L} m\nSpacing $s$ = {spacing:0.3f} m\nImpact parameter $b$\n{n}${{}}^3$ sensors\n{computed_n} random tracks', transform=ax.transAxes)
_= plt.show()


## Displacements in force law

In [ ]:
fig, ax = plt.subplots()
ax.bar(bins_b2[:-1],
       data_b2,
       width=np.diff(bins_b2),
       align="edge")

plt.loglog()
plt.axvline((spacing)**(-2), label='$s^{-2}$', color='red')
plt.legend()
plt.xlabel('$b^{-2}$ [m${}^{-2}$]')
plt.ylabel('Counts per line [arb.]')
plt.axhline(1, color='yellow', ls='--')
plt.xlim(*get_nonzero_xlim(data_b2, bins_b2))
plt.text(0.65, 0.35, f'Lattice side $L$ = {L} m\nSpacing $s$ = {spacing:0.3f} m\nImpact parameter $b$\n{n}${{}}^3$ sensors\n{computed_n} random tracks', transform=ax.transAxes)

_= plt.show()

## Impulse

In [ ]:
fig, ax = plt.subplots()
ax.bar(bins_J[:-1],
       data_J,
       width=np.diff(bins_J),
       align="edge")

plt.loglog()
plt.axvline(get_impulse(spacing), label='$s$', color='green')
plt.axvline(get_impulse(L), label='$L$', color='red')
plt.axhline(1, color='yellow', ls='--')
plt.legend()
plt.xlabel('Impulse [N$\cdot$s]')
plt.ylabel('Counts per line [arb.]')
plt.xlim(*get_nonzero_xlim(data_J, bins_J))
plt.text(0.65, 0.35, f'Lattice side $L$ = {L} m\nSpacing $s$ = {spacing:0.3f} m\nImpact parameter $b$\n{n}${{}}^3$ sensors\n{computed_n} random tracks', transform=ax.transAxes)

_= plt.show()